In [2]:
from pathlib import Path
import sys, inspect

PROJECT_ROOT = Path.cwd().parents[1]
sys.path.append(str(PROJECT_ROOT))
import dcf_models as dcf

print([n for n in dir(dcf) if not n.startswith("_")])
# If short, you can also peek:
# print(inspect.getsource(dcf))

# Try likely function names and alias them to the names you want to use:
dcf_npv = (
    getattr(dcf, "dcf_npv", None)
    or getattr(dcf, "npv", None)
    or getattr(dcf, "present_value", None)
    or getattr(dcf, "discounted_cash_flow", None)
)

['calculate_dcf']


In [4]:
# 1) Minimal NPV function (annual compounding; times in years)
def dcf_npv(cash_flows, r, times=None, freq=1):
    """
    cash_flows: list of CFs [CF1, CF2, ...]
    r: annual discount rate (decimal, e.g. 0.045)
    times: optional list of times in YEARS; default = 1..N
    freq: if CFs are per period (e.g., semiannual), set freq=2 and times in years
    """
    if times is None:
        times = range(1, len(cash_flows)+1)
    if freq == 1:
        return sum(cf / ((1 + r) ** t) for cf, t in zip(cash_flows, times))
    else:
        # per-period compounding when freq > 1
        return sum(cf / ((1 + r/freq) ** int(round(t*freq))) for cf, t in zip(cash_flows, times))

In [5]:
# 2) Example + save artifact
from pathlib import Path
import os

PROJECT_ROOT = Path.cwd().parents[1]
out_dir = PROJECT_ROOT / "outputs" / "04_DCF"
out_dir.mkdir(parents=True, exist_ok=True)

cash_flows = [10, 10, 10, 110]             # yearly coupon 10, redemption 100
npv = dcf_npv(cash_flows=cash_flows, r=0.045)

print(f"DCF NPV: {npv:.2f}")
with open(out_dir / "dcf_npv.txt", "w") as f:
    f.write(f"NPV={npv:.2f}")
print("✔ Saved:", out_dir / "dcf_npv.txt")

DCF NPV: 119.73
✔ Saved: /Users/katherinecohen/Documents/FixedIncomePortfolio/outputs/04_DCF/dcf_npv.txt
